# TALLER 10 - API TWITTER

In [1]:
##pip install tweepy
import tweepy
import pandas as pd

In [2]:
# Leer llaves
keys = pd.read_csv("keys_demo.csv", header=None)
keys = dict(zip(keys[0],keys[1]))

In [3]:
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_key_secret'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
tweets = tweepy.Cursor(api.search,
                           q=['johnny depp -RT'],
                       lang='es',
                       geocode=f'4.649178,-74.062827,52mi',
                       tweet_mode='extended',
                       include_rts=False).items(100)

In [5]:
t = []
for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogota")
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    t.append(temp)

In [6]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data

,tweet,date,city,user_location,id,link
0,"The people: Warner Bros, Bring Johnny Depp bac...",2020-11-07 17:05:42,Bogota,"Bogotá, D.C., Colombia",1325122266800160768,https://twitter.com/i/web/status/1325122266800...
1,"Alguien que me explique por qué Johnny Depp, d...",2020-11-07 16:08:10,Bogota,Bogotá,1325107785814716418,https://twitter.com/i/web/status/1325107785814...
2,Atención Cinéfilos | 🎬#ENTRETENIMIENTO | #CIN...,2020-11-07 15:03:38,Bogota,Bogotá,1325091545188610049,https://twitter.com/i/web/status/1325091545188...
3,Me niego ver #AnimalesFantasticos3 y a #acuama...,2020-11-07 12:14:03,Bogota,"Bogota, Colombia",1325048870385168384,https://twitter.com/i/web/status/1325048870385...
4,"Con la salida de Johnny Depp del cast, los com...",2020-11-07 10:43:51,Bogota,Bogotá - Colombia,1325026170547679233,https://twitter.com/i/web/status/1325026170547...
...,...,...,...,...,...,...
77,Calificado como “el mayor juicio por difamació...,2020-11-01 23:55:00,Bogota,"Bogotá, Colombia",1323050941671788548,https://twitter.com/i/web/status/1323050941671...
78,La artista de maquillaje Yuya Mika se transfor...,2020-11-01 18:21:28,Bogota,Bogota,1322967005876596737,https://twitter.com/i/web/status/1322967005876...
79,"Johnny Depp, en la búsqueda de que la justicia...",2020-11-01 15:57:05,Bogota,"Bogotá, Colombia",1322930672936407043,https://twitter.com/i/web/status/1322930672936...
80,Johnny Depp podría regresar como Homero en 'Lo...,2020-10-31 11:00:25,Bogota,Bogotá,1322493623762604032,https://twitter.com/i/web/status/1322493623762...


In [7]:
data = data[['tweet','date','city']]
data.head()

,tweet,date,city
0,"The people: Warner Bros, Bring Johnny Depp bac...",2020-11-07 17:05:42,Bogota
1,"Alguien que me explique por qué Johnny Depp, d...",2020-11-07 16:08:10,Bogota
2,Atención Cinéfilos | 🎬#ENTRETENIMIENTO | #CIN...,2020-11-07 15:03:38,Bogota
3,Me niego ver #AnimalesFantasticos3 y a #acuama...,2020-11-07 12:14:03,Bogota
4,"Con la salida de Johnny Depp del cast, los com...",2020-11-07 10:43:51,Bogota


In [8]:
import re
 
pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'
def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    
    for mention in mentions:
        text = text.replace(mention,"")
    return text, links, hashtags, mentions

In [9]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data.head()

C:\Users\cikan\Anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,tweet,date,city,clean,links,hashtags,mentions
0,"The people: Warner Bros, Bring Johnny Depp bac...",2020-11-07 17:05:42,Bogota,"The people: Warner Bros, Bring Johnny Depp bac...",[https://t.co/Ku63eI6Hdc],[],[@ChangeorgCO]
1,"Alguien que me explique por qué Johnny Depp, d...",2020-11-07 16:08:10,Bogota,"Alguien que me explique por qué Johnny Depp, d...",[],[],[]
2,Atención Cinéfilos | 🎬#ENTRETENIMIENTO | #CIN...,2020-11-07 15:03:38,Bogota,Atención Cinéfilos | 🎬#ENTRETENIMIENTO | | ...,"[https://t.co/yXFRJmf982, https://t.co/CnIFuKo...","[#CINE, #JohnnyDepp]",[]
3,Me niego ver #AnimalesFantasticos3 y a #acuama...,2020-11-07 12:14:03,Bogota,Me niego ver y a por la injusticia que esta ...,[],"[#AnimalesFantasticos3, #acuaman2]",[]
4,"Con la salida de Johnny Depp del cast, los com...",2020-11-07 10:43:51,Bogota,"Con la salida de Johnny Depp del cast, los com...",[],[],[]


In [10]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [11]:
data['fecha'] = data.date.dt.date
temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
temp

,fecha,cnt
5,2020-10-30,1
6,2020-10-31,1
4,2020-11-01,3
2,2020-11-02,22
3,2020-11-03,3
0,2020-11-06,29
1,2020-11-07,23


In [12]:
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'#1DA1F2'})
layout = go.Layout(title='Número de tuits por día')
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)